In [2]:
import requests
import re
from bs4 import BeautifulSoup, Comment

import json
import numpy as np
import pandas as pd
import time

from pymongo import MongoClient
from pprint import pprint
from connect import Connect

In [59]:
client = MongoClient()
database = client['NBA_tables']
collection = database['seasons']


In [78]:
res = collection.find({})
ret = list(res)
print(ret[0])

{'_id': ObjectId('5c644627d0ded788b0b8e3d2'), 'misc_tables': '<div class="table_wrapper setup_commented commented" id="all_misc_stats">\n<div class="section_heading">\n<span class="section_anchor" data-label="Miscellaneous Stats" id="misc_stats_link"></span><h2>Miscellaneous Stats</h2> <div class="section_heading_text">\n<ul>\n</ul>\n</div>\n</div>\n<div class="placeholder"></div>\n<!--\n   <div class="table_outer_container">\n      <div class="overthrow table_container" id="div_misc_stats">\n  <table class="sortable stats_table" id="misc_stats" data-cols-to-freeze=2><caption>Miscellaneous Stats Table</caption>\n   <colgroup><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col></colgroup>\n   <thead>\n      \n      <tr class="over_header">\n         <th aria-label="" data-stat="" colspan="17" class=" over_header center" ></th>\n         <th aria-label="" data-stat="header_team" colspan="4" class="bol

In [18]:
season_2001 = ['https://www.basketball-reference.com/leagues/NBA_2001.html', 'https://www.basketball-reference.com/leagues/NBA_2002.html']



In [44]:
def scrape(season_urls):
    season = 2001
    for i in season_urls:
        fields= {}
        webpage = requests.get(i)
        soup = BeautifulSoup(webpage.text, 'lxml')
        fields['misc_tables'] = str(soup.find_all('div', id="all_misc_stats")[0])
        fields['team_stats_table'] = str(soup.find_all('div', id='all_team-stats-base')[0])
        fields['shooting_table'] = str(soup.find_all('div', id='all_team_shooting')[0])

        fields['season'] = season
        season +=1
        return fields
        
        

In [45]:
mg_01_02 = scrape(season_2001)

In [46]:
mg_01_02

{'misc_tables': '<div class="table_wrapper setup_commented commented" id="all_misc_stats">\n<div class="section_heading">\n<span class="section_anchor" data-label="Miscellaneous Stats" id="misc_stats_link"></span><h2>Miscellaneous Stats</h2> <div class="section_heading_text">\n<ul>\n</ul>\n</div>\n</div>\n<div class="placeholder"></div>\n<!--\n   <div class="table_outer_container">\n      <div class="overthrow table_container" id="div_misc_stats">\n  <table class="sortable stats_table" id="misc_stats" data-cols-to-freeze=2><caption>Miscellaneous Stats Table</caption>\n   <colgroup><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col></colgroup>\n   <thead>\n      \n      <tr class="over_header">\n         <th aria-label="" data-stat="" colspan="17" class=" over_header center" ></th>\n         <th aria-label="" data-stat="header_team" colspan="4" class="bold_text over_header center" >Offense Four Fact

In [47]:
if mg_01_02['season'] == 2001:
    misc_tables = mg_01_02['misc_tables']

In [48]:
soup = BeautifulSoup(str(misc_tables), 'lxml') 

In [49]:
def misc_maker(html):
    misc = []
    for comment in soup.find_all(string=lambda text:isinstance(text,Comment)):
        data = BeautifulSoup(comment,"lxml")
        for items in data.select("table.stats_table tr"):
            tds = [item.get_text(strip=True) for item in items.select("th,td")]
            misc.append(tds)
    miscar = pd.DataFrame(misc[2:], columns= misc[1])       
    miscar

In [41]:
miscar_small = miscar[['Rk', 'Team', 'W', 'ORtg', 'DRtg', 'Pace', '3PAr', 'TS%']]
miscar_small

,Rk,Team,W,ORtg,DRtg,Pace,3PAr,TS%
0,1,Sacramento Kings*,61,109.0,101.1,95.6,.166,.539
1,2,Los Angeles Lakers*,58,109.4,101.7,92.1,.210,.534
2,3,San Antonio Spurs*,58,106.5,99.7,90.0,.190,.539
3,4,Dallas Mavericks*,57,112.2,107.7,92.8,.237,.553
4,5,New Jersey Nets*,52,104.0,99.5,91.8,.175,.515
5,6,Minnesota Timberwolves*,50,109.0,105.3,90.5,.152,.532
6,7,Seattle SuperSonics*,45,108.9,105.6,89.0,.193,.540
7,8,Portland Trail Blazers*,49,107.4,104.0,89.3,.198,.528
8,9,Boston Celtics*,49,103.4,101.0,92.5,.289,.520
9,10,Detroit Pistons*,50,104.8,102.4,90.0,.240,.540


In [ ]:
team_stats = []
for comment in team_stats_table.find_all(string=lambda text:isinstance(text,Comment)):
    data = BeautifulSoup(comment,"lxml")
    for items in data.select("table.stats_table tr"):
        tds = [item.get_text(strip=True) for item in items.select("th,td")]
        team_stats.append(tds)
ts = pd.DataFrame(team_stats[1:], columns= team_stats[0])       
ts

In [ ]:
shoot_stats = []
for comment in shooting_table.find_all(string=lambda text:isinstance(text,Comment)):
    data = BeautifulSoup(comment,"lxml")
    for items in data.select("table.stats_table tr"):
        tds = [item.get_text(strip=True) for item in items.select("th,td")]
        shoot_stats.append(tds)
#ss = pd.DataFrame(shoot_stats[1:], columns= shoot_stats[0])    
ss = pd.DataFrame(shoot_stats[3:], columns = shoot_stats[2])
#ss = pd.DataFrame(shoot_stats)
ss